In [ ]:
import sys
sys.path.append("/home/mengjingliu/ADL_unsupervised_learning/")
from ReprogramLLM.data_loader import load_dataFile, resize_tensor
import matplotlib.pyplot as plt
import torch

In [ ]:
def plot_doppler(data, title=""):
    plt.figure(figsize=(8, 6))  # Set the figure size as desired
    plt.imshow(data, cmap='hot', interpolation='nearest')
    plt.colorbar()  # Show color scale
    plt.title(title)
    plt.show()

In [ ]:
path_list = ["/home/mengjingliu/Vid2Doppler/data/2023_07_19/HAR6",
				"/home/mengjingliu/Vid2Doppler/data/2023_07_19/HAR5",
				"/home/mengjingliu/Vid2Doppler/data/2023_07_19/HAR4",
				"/home/mengjingliu/Vid2Doppler/data/2023_11_17/HAR3",
				"/home/mengjingliu/Vid2Doppler/data/2023_07_19/HAR2"]

train_data, train_label, test_data, test_label = load_dataFile(path_list, train_size=0.8)
train_data_re, test_data_re = resize_tensor(train_data, test_data)

train_data, test_data = train_data.numpy(), test_data.numpy()
train_data_re, test_data_re = train_data_re.numpy(), test_data_re.numpy()

In [ ]:
for i in range(10):
    plot_doppler(train_data[i][0], title="original")
    plot_doppler(train_data_re[i][0], title="resized")


In [5]:
# test ViT
import sys
sys.path.append("/home/mengjingliu/ADL_unsupervised_learning/")
import datetime
from torchmetrics import Accuracy
from ReprogramLLM.data_loader import wrapper_dataLoader
from transformers import ViTForImageClassification
import torch
from torch.optim import Adam
from transformers import get_scheduler
from tqdm.auto import tqdm
from torch.optim.lr_scheduler import ExponentialLR
import numpy as np
from torchsummary import summary
import torch.nn as nn
import logging
import os

In [ ]:
# load model from .pth
model_name_or_path = "google/vit-base-patch16-224-in21k"
title = model_name_or_path.replace('/', '_') + "_standard"
# Load a pre-trained Vision Transformer model
model = ViTForImageClassification.from_pretrained(model_name_or_path, num_labels=5)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Move model to device
model.to(device)
# Load the state dict
state_dict = torch.load('/home/mengjingliu/ADL_unsupervised_learning/ViT/results/best_model_smallViT_google_vit-base-patch16-224-in21k.pth')

# Load the state dict into the model
model.load_state_dict(state_dict)

In [15]:
# load small ViT
model_name_or_path = "google/vit-base-patch16-224-in21k"
title = model_name_or_path.replace('/', '_') + "_standard"
# Load a pre-trained Vision Transformer model

from transformers import ViTConfig

# config a smaller ViT.
configuration = ViTConfig(
    num_labels=5,
    # hidden_dropout_prob = 0.2,
    # attention_probs_dropout_prob = 0.2
    hidden_size=128,
    num_hidden_layers=4,
    num_attention_heads=4,
    intermediate_size=128,
    num_channels=1,
)

# Load a pre-trained Vision Transformer model
model = ViTForImageClassification.from_pretrained(model_name_or_path, config=configuration, ignore_mismatched_sizes=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Move model to device
model.to(device)
# Load the state dict
state_dict = torch.load('/home/mengjingliu/ADL_unsupervised_learning/ViT/results/best_model_smallViT_google_vit-base-patch16-224-in21k.pth')

# Load the state dict into the model
model.load_state_dict(state_dict)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized because the shapes did not match:
- embeddings.cls_token: found shape torch.Size([1, 1, 768]) in the checkpoint and torch.Size([1, 1, 128]) in the model instantiated
- embeddings.patch_embeddings.projection.bias: found shape torch.Size([768]) in the checkpoint and torch.Size([128]) in the model instantiated
- embeddings.patch_embeddings.projection.weight: found shape torch.Size([768, 3, 16, 16]) in the checkpoint and torch.Size([128, 1, 16, 16]) in the model instantiated
- embeddings.position_embeddings: found shape torch.Size([1, 197, 7

<All keys matched successfully>

In [12]:
# Load LoRA
# load model from .pth
model_name_or_path = "google/vit-base-patch16-224-in21k"
title = model_name_or_path.replace('/', '_') + "_standard"
# Load a pre-trained Vision Transformer model
model = ViTForImageClassification.from_pretrained(model_name_or_path, num_labels=5)

from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"],
)
model = get_peft_model(model, config)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Move model to device
model.to(device)
# Load the state dict
state_dict = torch.load('/home/mengjingliu/ADL_unsupervised_learning/ViT/results/best_model_google_vit-base-patch16-224-in21k_LoRA.pth')

# Load the state dict into the model
model.load_state_dict(state_dict)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [8]:
# load test data from .npy
import torch.nn.functional as F
X_test = np.load("/home/mengjingliu/ADL_unsupervised_learning/ViT/X_test.npy")
y_test = np.load("/home/mengjingliu/ADL_unsupervised_learning/ViT/y_test.npy")

X_test, y_test = torch.from_numpy(X_test).float(), torch.from_numpy(y_test).long()
X_test = F.interpolate(X_test, size=(224, 224), mode='bilinear', align_corners=False)
# X_test = X_test.repeat(1, 3, 1, 1)

In [16]:
# evaluate model 
model.eval()
with torch.no_grad():
    inputs = X_test.to(device)
    outputs = model(inputs).logits.to("cpu")

accuracy = Accuracy(task="multiclass", num_classes=5).to(device)
accuracy = accuracy(outputs, y_test)
print(f'Testing accuracy: {accuracy}')


Testing accuracy: 0.8481675386428833
